In [8]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

def clean_instagram_data(file_path):
    # 1. Baca data
    df = pd.read_excel(file_path, sheet_name='Sheet1')
    
    # 2. Hapus baris yang seluruhnya kosong
    df = df.dropna(how='all')
    
    # 3. Hapus kolom yang tidak diinginkan
    df = df.drop(columns=['content_category', 'media_url'])
    
    # 4. Bersihkan kolom likes
    df['likes'] = df['likes'].astype(str).str.replace(',', '').astype(int)
    
    # 5. Konversi kolom waktu
    df['upload_time'] = pd.to_datetime(df['upload_time']).dt.tz_localize(None)
    
    # 6. Fungsi untuk membersihkan komentar
    def clean_comments(comment_text):
        if not isinstance(comment_text, str):
            return ""
        
        patterns = [
            r'June \d+|May \d+|2025',
            r'Meta|About|Blog|Jobs|Help|API|Privacy|Terms|Locations|batikula|Original|audio',
            r'Instagram Lite|Threads|Contact Uploading & Non-Users',
            r'Meta Verified|Meta in Indonesia|English',
            r'© \d+ Instagram from Meta',
            r'Home|Search|Explore|Reels|Messages|Notifications',
            r'Create|Dashboard|Profile|Meta AI|AI Studio|More'
        ]
        
        for pattern in patterns:
            comment_text = re.sub(pattern, '', comment_text)
        
        comment_text = re.sub(r'[^\w\s.,!?]', '', comment_text)
        comment_text = ' '.join(comment_text.split())
        
        return comment_text
    
    df['all_comments'] = df['all_comments'].apply(clean_comments)
    
    
    # 8. Ekstrak hashtag dan format dengan pagar
    def extract_hashtags(caption):
        if not isinstance(caption, str):
            return ""
        
        hashtags = re.findall(r'#(\w+)', caption)
        # Gabungkan dengan pagar dan pisahkan dengan spasi
        return ' '.join([f'#{tag}' for tag in hashtags])
    
    df['hashtags'] = df['caption'].apply(extract_hashtags)
    
    # 9. Tambahkan kolom analisis waktu
    df['post_hour'] = df['upload_time'].dt.hour
    df['post_day'] = df['upload_time'].dt.day_name()
    
    # 10. Urutkan berdasarkan tanggal
    df = df.sort_values('upload_time', ascending=False)
    
    return df

# Jalankan fungsi cleaning
try:
    cleaned_data = clean_instagram_data('konten_ig_Batikula_baru.xlsx')
    
    # Simpan hasil
    cleaned_data.to_excel('batikula_cleaned_complete.xlsx', 
                         index=False, 
                         engine='openpyxl')
    cleaned_data.to_csv('batikula_cleaned_complete.csv', index=False)
    
    print("Pembersihan data selesai. File tersimpan sebagai:")
    print("- batikula_cleaned_complete.xlsx")
    print("- batikula_cleaned_complete.csv")
    
except Exception as e:
    print(f"Terjadi error: {str(e)}")

Pembersihan data selesai. File tersimpan sebagai:
- batikula_cleaned_complete.xlsx
- batikula_cleaned_complete.csv
